# Importing Packages

In [1]:
import csv
import random
import numpy as np
import pandas as pd
import string
from datetime import datetime, timedelta
from faker import Faker
import os
from geopy.distance import geodesic
import math

In [5]:
%cd C:\Users\shrav\Data_Analysis_Projects\Big Projects\Project Dream Air\Datasets

C:\Users\shrav\Data_Analysis_Projects\Big Projects\Project Dream Air\Datasets


# Initialize Faker

In [2]:
fake = Faker()

# roundup()

In [3]:
def roundup(x):
    return math.ceil(x / 10.0) * 10

# return_time_from_decimal()

In [4]:
def return_time_from_decimal(decimal_time: int):
    '''
    Returns a HH:MM:SS(seconds = 0) format time from a decimal format of HH.MM
    :param decimal_time: time format of HH.MM
    '''
    f_hour = int(decimal_time)
    f_minute = round((decimal_time - f_hour) * 60)
    return f'{f_hour:02}:{f_minute:02}:00'

# routes_dict_generator()

In [5]:
def routes_dict_generator(input_file: str) -> dict:
    """Creates a dict {route_id: [price, travel_time]}
    :param input_file: Must be in the format of (ID,Source,Destination,Price,Travel_time)
    :return: {route_id: [price, travel_time]}"""
    with open(input_file, encoding='utf-8', newline='') as routes_file:
        routes_file.readline().strip('\n\r').split(',')
        routes_reader = csv.reader(routes_file)
        return {int(f_row[0]): f_row[3:] for f_row in routes_reader}

# aircraft_dict_generator()

In [6]:
def aircraft_dict_generator(input_file: str) -> dict:
    """Creates a dict {aircraft_id: seats}
    :param input_file: Must be in the format (aircraft_ID,aircraft_reg_no,aircraft_name,seats)
    :return: {aircraft_id: seats}"""
    with open(input_file, encoding='utf-8', newline='') as f_aircraft_file:
        f_aircraft_file.readline().strip('\n\r').split(',')
        aircraft_file_reader = csv.reader(f_aircraft_file)
        return {int(f_row[0]): f_row[3] for f_row in aircraft_file_reader}

# round_off_time()

In [7]:
def round_off_time(time_string: str):
    '''
    Returns a time string of "%H:%M:%S" where the minutes are rounded off to nearest (0, 15, 30, 45) min.
    If minute part is over 45 then the next hour is returned.
    
    time_string: Time string in a format HH:MM:SS
    
    return Rounded off time string to the nearest (0, 15, 30, 45)
    '''
    time_obj = datetime.strptime(time_string, "%H:%M:%S")
    available_minutes = (0, 15, 30, 45)
    curr_minute = time_obj.minute
    
    if curr_minute in available_minutes:
        return time_string

    next_minute = min((minute for minute in available_minutes if minute > curr_minute), default=0)
    
    if next_minute == 0:
        time_obj = time_obj.replace(minute=0, second=0) + timedelta(hours=1)
    else:
        time_obj = time_obj.replace(minute=next_minute, second=0)
        
    rounded_time = time_obj.strftime("%H:%M:%S")

    return rounded_time

In [8]:
da_aircraft_speed = 804.67  # km/hr
da_price_multiplier = 4.5  # Rs/km
da_airport_coordinates = {
    "DEL": [(28.5562, 77.1000), 1],  # Indira Gandhi International Airport, Delhi
    "BOM": [(19.0896, 72.8656), 1],  # Chhatrapati Shivaji Maharaj International Airport, Mumbai
    "BLR": [(13.1986, 77.7066), 1],  # Kempegowda International Airport, Bangalore
    "HYD": [(17.2288, 78.4292), 2],  # Rajiv Gandhi International Airport, Hyderabad
    "MAA": [(12.9942, 80.1756), 2],  # Chennai International Airport, Chennai
    "CCU": [(22.6559, 88.8425), 2],  # Netaji Subhas Chandra Bose International Airport, Kolkata
    "AMD": [(23.1167, 72.6348), 2],  # Sardar Vallabhbhai Patel International Airport, Ahmedabad
    "COK": [(10.1523, 76.4017), 3],  # Cochin International Airport, Kochi
    "GOI": [(15.3809, 74.1487), 3],  # Dabolim Airport, Goa
    "LKO": [(26.7606, 80.8890), 3],  # Chaudhary Charan Singh International Airport, Lucknow
    "GAU": [(26.0334, 91.5880), 3],  # Lokpriya Gopinath Bordoloi International Airport, Guwahati
    "JAI": [(26.8250, 75.8113), 4],  # Jaipur International Airport, Jaipur
    "BBI": [(20.2985, 85.8189), 4],  # Biju Patnaik International Airport, Bhubaneswar
    "SXR": [(33.9985, 74.1795), 4]   # Srinagar International Airport, Srinagar
}

# create_routes()

In [9]:
def create_routes(output_file: str, airport_coordinates: dict, aircraft_speed: float, price_multiplier: float, convenience_fee: int) -> None:
    """
    Uses airport_coordinates dict to calculate distance between the two airports, calculates time with the 
    aircraft_speed and calculates price using price_multiplier(rs/km) and an additional convinence fee.
    o/p csv will be like this = (ID, Source, Destination, Distance, Travel_Time, Popularity, Price) The Source and Destination will be foriegn keys for the primary key of Airports table.
    
    :param output_file: File name where the csv data needs to go.
    :param airport_coordinates: Dict with aiport IATA Code as keys and a list of coordinates and aiport popularity {IATA Code: [coordinates, popularity]}
    :param aircraft_speed: Aircraft speed (Km/hr)
    :param price_multiplier: Price per Km 
    :param convenience_fee: standard convenience_fee for the airline company

    :rtype: None
    """
    list_of_airports = list(airport_coordinates.keys())
    no_of_airports = len(airport_coordinates) + 1
    routes_list = []
    
    for source_index in range(1, no_of_airports):
        for destination_index in range(source_index + 1, no_of_airports):
            source = list_of_airports[source_index - 1]
            destination = list_of_airports[destination_index - 1]
            
            distance = geodesic(airport_coordinates[source][0], airport_coordinates[destination][0]).kilometers
            time = round((distance / aircraft_speed) + 0.5, 2)  # (0.5) take off and landing
            
            formatted_time = return_time_from_decimal(time)
            rounded_off_time = round_off_time(formatted_time)

            popularity = [airport_coordinates[source][1], airport_coordinates[destination][1]]
            price = (distance * price_multiplier) + convenience_fee

            if price < 2500:
                price = 3000
            elif price < 3500:
                price = 3750

            source_list = [source_index, destination_index, roundup(distance), rounded_off_time, sum(popularity) / 2, round(price, 2)]
            destination_list = [destination_index, source_index, roundup(distance), rounded_off_time, sum(popularity) / 2, round(price, 2)]

            routes_list.append(source_list)
            routes_list.append(destination_list)

    df = pd.DataFrame(routes_list, 
                      columns=["Source", "Destination", "Distance", "Travel_Time", "Popularity", "Price"])
    
    df.sort_values(by="Popularity", inplace=True)
    df.drop("Popularity", axis=1, inplace=True)
    df.index = range(1, len(df) + 1)
    df = df.rename_axis(index='ID')
    df.to_csv(output_file)


# ticket_estimator()

In [10]:
def ticket_estimator(f_date: str, hike_list: list, tickets: int, f_year) -> int:
    """ Generated a ticket estimate for the given f_year (for my use, intentionally used to depict a change in orders
    to simulate a real dataset"""
    year_adjustments = {2018: -5, 2020: -20}
    tickets += year_adjustments.get(f_year, 0)

    day_name = day_of_week_generator(f_date)

    if f_date[5:] in hike_list:
        if day_name not in ("Thu", "Tue", "Wed"):
            return tickets
        elif day_name == "Wed":
            reduction = random.randint(10, 15)
        else:
            reduction = random.randint(5, 10)
            
    else:
        if day_name in ("Thu", "Tue"):
            reduction = random.randint(10, 20)
        elif day_name == "Wed":
            if f_year == 2018 or f_year == 2020:
                reduction = random.randint(25, 35)
            reduction = random.randint(10, 25)
        else:
            reduction = random.randint(0, 10)
            
    return int(tickets - tickets * (reduction/100))

# generate_random_order_date()

In [11]:
def generate_random_order_date(date_time: str):
    """Creates a random datetime(randomly selected between 1 - 90 days, time is also random) before the given date
    :param: str of format yyyy-mm-dd hh-mm-ss
    :return: generates date in format of (yyyy-mm-dd hh-mm-ss) 90 days before the given date_time"""

    random_days = random.randint(1, 90)
    random_hours = random.randint(0, 23)
    random_minutes = random.randint(0, 59)
    random_seconds = random.randint(0, 59)

    date_obj = datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')
    
    # Subtract the random number of days from the base date
    random_date = date_obj - timedelta(days=random_days)
    
    # Subtract the random time from the base date
    random_datetime = random_date - timedelta(hours=random_hours, minutes=random_minutes, seconds=random_seconds)
    formatted_date = random_datetime.strftime('%Y-%m-%d %H:%M:%S')
    
    return formatted_date


# day_of_week_generator()

In [12]:
def day_of_week_generator(date_str: str) -> str:
    date_object = datetime.strptime(date_str, "%Y-%m-%d")
    
    return date_object.strftime("%a")

# return_date_list()

In [13]:
def return_date_list(year: int) -> list:
    start_date = str(year) + "-01-01"
    end_date = str(year) + "-12-31"
    date_range = pd.date_range(start=start_date, end=end_date)
    
    return date_range.strftime('%Y-%m-%d').to_list()
    

# hike_dates_list_generator

In [14]:
def hike_dates_list_generator(input_file: str) -> list:
    """Creates a list of date(mm-dd) of hike dates
    :param input_file: must be in a format of [date1, date2, ...dateN]
    :return: list of date(mm-dd)"""

    hike_dates = pd.read_csv(input_file)
    return hike_dates['Hike-dates(mm-dd)'].tolist()
    

# generate_seat_numbers()

In [15]:
def generate_seat_numbers(f_seats: int, f_columns=6) -> list:
    '''
    Generate f_seats amount of seats like shown
    1A 1B 1C 1D 1E 1F 2A 2B 2C 2D 2E 2F .  .

    :param f_seats: Total amount of seats in your aeroplane.
    :param f_columns: Total amount of columns (A, B, C, ...) default 6
    '''
    seats = []
    seat_letters = list(string.ascii_uppercase[:f_columns])  # A-F columns
    row = 1

    while len(seats) < f_seats:
        for letter in seat_letters:
            seats.append(f"{row}{letter}")
            if len(seats) == f_seats:
                break  # Stop once we have 160 seats
        row += 1
    return seats

# generate_random_order_date()

In [16]:
def generate_random_order_date(date_time: str):
    """Creates a random datetime(randomly selected between 1 - 90 days, time is also random) before the given date
    :param: str of format yyyy-mm-dd hh-mm-ss
    :return: generates date in format of (yyyy-mm-dd hh-mm-ss) 90 days before the given date_time"""

    random_days = random.randint(1, 90)
    random_hours = random.randint(0, 23)
    random_minutes = random.randint(0, 59)
    random_seconds = random.randint(0, 59)

    date_obj = datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')
    # Subtract the random number of days from the base date
    random_date = date_obj - timedelta(days=random_days)
    # Subtract the random time from the base date
    random_datetime = random_date - timedelta(hours=random_hours, minutes=random_minutes, seconds=random_seconds)
    formatted_date = random_datetime.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_date

# route_for_day_of_week()

In [17]:
def route_for_day_of_week(day_of_week: str, schedule_day: int) -> tuple:
    if day_of_week == 'Sat':
        list_routes_of_day = saturday_route_schedule
        list_flights_of_day = saturday_flight_schedule
        list_departure_time = weekend_departure_times

    elif day_of_week == 'Sun':
        list_routes_of_day = sunday_route_schedule
        list_flights_of_day = sunday_flight_schedule
        list_departure_time = weekend_departure_times

    else:
        list_routes_of_day = route_schedule_data[schedule_day]
        list_flights_of_day = flight_schedule_data[schedule_day]
        list_departure_time = weekday_departure_times

    return list_routes_of_day, list_flights_of_day, list_departure_time

# fake_phone_number()

In [18]:
def fake_phone_number(f_fake: Faker) -> str:
    """
    Creates a fake phone number with +91 at teh start
    :param f_fake: fake
    :return: +91 10_digit_ph_no
    """
    return f'+91 {f_fake.msisdn()[3:]}'

# return_date_list()

In [19]:
def return_date_list(year: int) -> list:
    start_date = str(year) + "-01-01"
    end_date = str(year) + "-12-31"
    date_range = pd.date_range(start=start_date, end=end_date)
    return date_range.strftime('%Y-%m-%d').to_list()

# orders_generator()

In [20]:
def orders_generator(customer_limit: int,
                     year: int,
                     output_file: str,
                     schedule_output_file: str,
                     customer_start: int = 0,
                     orders_id: int = 1) -> tuple:
    """
    Creates a csv file with format
    (order_id,customer_id,route_id,flight_id,order_date,departure_time,base_price,seat_no)\n

    :param schedule_output_file: creates a new file with date and the route_id based on the incomplete files
    :param customer_limit: selects a random number from (1, customer_limit) for the customer_id
    :param year: generates a list of dates of the given year and uses the year to generate a ticket estimate
    :param output_file: file to print the data
    :return: orders_id
    """
    
    schedule_list = []
    orders_list = []

    # date_count needs to be incremented by 1 as my schedule file has monday to friday 
    date_count = 0
    if year == 2019:
        date_count = 1
    elif year == 2020:
        date_count = 2
    elif year == 2021:
        date_count = 3


    # orders_id takes on the last order_id from the previous iteration as to maintain contiguous primary key
    order_id = orders_id
    schedule_id = 1

    # returns the list of dates in that year
    list_of_dates = return_date_list(year)

    # Now iterating through the newly generated list of dates
    for date in list_of_dates:

        # in my routes schedule file I have 59 routes when we reach the last row i wanted to get the row counter back to 
        # the first row thats y date_count = 0 when it hits 60
        if date_count == 60: 
            date_count = 0

        # Get a day name for each iteration
        day_name = day_of_week_generator(date)

        # returns the routes, flights and the departure dates for that day
        routes_of_day, flights_of_day, departure_time_list = route_for_day_of_week(day_name, date_count)
        number_of_routes = len(routes_of_day)

        # Now iterate through the routes present in the current day
        for route in range(number_of_routes):
            # route id and flight id will be the current route
            route_id = routes_of_day[route]
            f_flight_id = flights_of_day[route]
            # Creating Departure date datetime
            departure_date = date + " " + departure_time_list[route]

            # Getting the base price 
            base_price = float(all_routes_dict[route_id][-1])

            # Year wise discount or increase based on personal preference
            if year == 2018:
                base_price -= base_price * 0.1
            elif year == 2019:
                base_price += base_price * 0.2

            # Calculate the amount of seats present in the current flight
            max_tickets = int(aircraft_dict[f_flight_id])
            
            ticket_estimate = ticket_estimator(date, hike_dates, max_tickets, year)
            total_seats = generate_seat_numbers(max_tickets)

            # Create schedules file with this list
            schedules_row = [schedule_id, date, route_id, f_flight_id, departure_time_list[route]]
            schedule_list.append(schedules_row)

            # Generate order dates for all the ticktes before the departure date ans then sort it 
            new_order_dates = [generate_random_order_date(departure_date) for _ in range(ticket_estimate)]
            new_order_dates.sort()
            
            schedule_id += 1

            for seat in range(ticket_estimate):
                # Select a random seat and the earliest order date
                random_seat = random.choice(total_seats)
                order_date = new_order_dates[seat]

                # Select a random customer from these limit to mimic a customer
                start = customer_start + 1
                end = customer_limit + customer_start
                customer_id = random.randint(start, end)
                
                orders_row = [order_id, customer_id, route_id, f_flight_id, order_date, departure_date, base_price, random_seat]
                orders_list.append(orders_row)

                # once the seat is added to the list remove the seat from the list of seats 
                total_seats.remove(random_seat)
                
                order_id += 1
                
        if day_name not in ("Sat", "Sun"):
            date_count += 1

    df_schedules = pd.DataFrame(schedule_list, 
                                columns=['schedule_id', 'date', 'route_id', 'flight_id', 'departure_time'],
                                index=range(1, len(schedule_list) + 1))
    df_schedules.to_csv(schedule_output_file, index=False)
    del schedule_list, df_schedules

    df_orders = pd.DataFrame(orders_list, 
                            columns=['order_id', 'customer_id', 'route_id', 'flight_id', 'order_date', 'departure_time', 'base_price', 'seat_no'],
                            index=range(1, len(orders_list) + 1))
    df_orders.to_csv(output_file, index=False)
    del orders_list, df_orders

    # return the last order id and the total customer id to facilitate the next iteration
    return order_id, end
    

# cancellations_updator()

In [21]:
def cancellations_updator(f_input_file: str, f_output_file: str, cancellation_percentage: int,
                          customer_limit: int, customer_start: int = 0, f_id: int = 1) -> tuple:
    """
    updates the input file with a cancellations column based on the percentage given with `cancellation_percentage` of
    records will show "Cancelled" and creates an output file with all the "Cancelled" orders(no index in the output_file)
    :param customer_limit:
    :param f_input_file: any `csv` file
    :param f_output_file: any file
    :param cancellation_percentage: percentage of records to show "Cancelled" at the end
    :return: None
    """

    # Cancelling some orders randomly
    df = pd.read_csv(f_input_file)
    total_records = len(df)
    cancellations_entries = int(total_records * (cancellation_percentage / 100))
    no_indices = random.sample(range(total_records), cancellations_entries)
    df['cancellations'] = "Confirmed"
    df.loc[no_indices, 'cancellations'] = 'Cancelled'
    df.to_csv('temporary_file.csv', index=False)

    new_order_id = f_id
    with (open('temporary_file.csv', encoding='utf-8', newline='') as orders_c_file,
          open(f_input_file, 'w', newline='') as orders_final_file,
          open(f_output_file, 'w', newline='') as cancellations_file):
        orders_c_file.readline().strip('\r\n').split(',')
        reader = csv.reader(orders_c_file)
        print('order_id,customer_id,route_id,flight_id,order_date,departure_date,base_price,seat_no,Confirmation',
              file=orders_final_file)
        print('order_id,cancellation_date',
              file=cancellations_file)
        for row in reader:
            order_id, customer_id, route_id, flight_id, order_date, departure_date, base_price, seat_no, cancellations = row
            
            if cancellations == 'Cancelled':
                start = customer_start + 1
                end = customer_limit + customer_start
                new_customer_id = random.randint(start, end)

                # get a cancellation date which is after the confirmed date and before the departure date
                cancellation_date = generate_random_datetime(order_date, departure_date)
                cancellation_date_str = cancellation_date.strftime("%Y-%m-%d %H:%M:%S")
                new_order_date = generate_random_datetime(cancellation_date_str, departure_date)
                                                         
                print(f'{new_order_id},{customer_id},{route_id},{flight_id},{order_date},{departure_date},{base_price},{seat_no},{cancellations}', 
                  file=orders_final_file)
                                                        
                print(
                    f'{new_order_id},{cancellation_date}',
                        file=cancellations_file)
                new_order_id += 1
                
                print(
                    f'{new_order_id},{new_customer_id},{route_id},{flight_id},{new_order_date},{departure_date},{base_price},{seat_no},Confirmed',
                    file=orders_final_file)
                                                          
                new_order_id += 1
                
            else:    
                print(f'{new_order_id},{customer_id},{route_id},{flight_id},{order_date},{departure_date},{base_price},{seat_no},{cancellations}', 
                      file=orders_final_file)
                new_order_id += 1
    os.remove('temporary_file.csv')

    return new_order_id, end


# sort_by_order_date()

In [22]:
def sort_by_order_date(input_file: str) -> None:
    df = pd.read_csv(input_file)
    df = df.sort_values(by='order_date')
    df.reset_index(drop=True, inplace=True)
    df.index = df.index + 1
    df = df.drop(columns=['order_id'])
    df.to_csv(input_file, index=True, index_label='order_id')

# customer_membership_status()

In [23]:
def customer_membership_status(input_file: str, percentage: int) -> None:
    """
    Updates the input_file with membership status based on the percentage given
    :param input_file: customers_file
    :param percentage: percentage of record to show 'Yes' in the membership column
    :return: None
    """
    df = pd.read_csv(input_file)
    num_records = len(df)
    sample_size = int((percentage / 100) * num_records)
    yes_indices = random.sample(range(num_records), sample_size)
    df['membership'] = 'NO'
    df.loc[yes_indices, 'membership'] = 'YES'
    df.to_csv(input_file, index=False)

# generate_random_age()

In [24]:
def generate_random_age():
    # Define the age ranges and their corresponding weights
    age_ranges = [(25, 50), (12, 80)]
    weights = [0.8, 0.2]  # 80% chance for (25, 50), 20% for (12, 80)
    
    # Choose a range based on the defined weights
    chosen_range = random.choices(age_ranges, weights, k=1)[0]
    
    # Generate a random age within the chosen range
    random_age = random.randint(chosen_range[0], chosen_range[1])
    return random_age

# customer_generator()

In [25]:
def customer_generator(input_file: str, customer_limit: int, fc_id: int = 1) -> int:
    """Generates customer_id, f_name, l_name, ph_no and email using faker for the desired customer_limit
    Don't use a large number as it might take a long time
    :param input_file: file to generate the values into
    :param customer_limit: amount of records to be printed in that file
    :return: None"""
    with open(input_file, 'w', newline='') as customer_file:
        c_id = fc_id
        print("c_id,name,gender,age,ph_no,email", file=customer_file)
        for _ in range(customer_limit):
            genders = ['Male', 'Female']
            gender = fake.random_element(genders)
            name = fake.name_male() if gender == "Male" else fake.name_female()
            ph_no = fake_phone_number(fake)
            email = name + ph_no[-3:] + "@gmail.com"
            age = generate_random_age()
            print(f"{c_id},{name},{gender},{age},{ph_no},{email}", file=customer_file)
            c_id += 1
    return c_id

# generate_random_datetime()

In [26]:
def generate_random_datetime(start_date: str, end_date: str):
    """
    Generates a random datetime value between the start_date and end_date
    :param start_date: date_time str
    :param end_date: date_time str
    :return: Return a datetime that is between the start_date and end_date
    """
    start_dt = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    time_difference = (end_dt - start_dt).total_seconds()
    random_seconds = random.randint(0, int(time_difference))
    return start_dt + timedelta(seconds=random_seconds)

In [27]:
df_route_schedule = pd.read_csv("routes_schedule_complete.csv", header=None)
route_schedule_data = df_route_schedule.to_records(False).tolist()
saturday_route_schedule = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
sunday_route_schedule = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

df_flight_schedule = pd.read_csv("flight_schedule_complete.csv", header=None)
flight_schedule_data = df_flight_schedule.to_records(False).tolist()
saturday_flight_schedule = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
sunday_flight_schedule = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

weekday_departure_times = ['06:00:00', '07:30:00', '08:00:00', '09:00:00', '10:30:00', '11:00:00', '12:00:00',
                           '13:00:00', '14:30:00', '15:30:00', '17:30:00', '18:00:00', '18:30:00', '19:30:00',
                           '21:30:00']
weekend_departure_times = ['06:00:00', '06:30:00', '07:00:00', '07:30:00', '08:00:00', '09:00:00', '10:30:00',
                           '11:00:00', '12:00:00', '13:00:00', '14:30:00', '15:30:00', '17:30:00', '18:00:00',
                           '18:30:00', '19:30:00', '20:30:00', '21:30:00']

# Generating job roles and both ground and air staffs 

In [6]:
aircraft_file = pd.read_csv("DreamAir_Aircraft.csv")
flight_id_list = aircraft_file["aircraft_ID"].tolist()

with (open("DreamAir_Job_Roles.csv", encoding='utf-8', newline='') as job_roles_file,
      open("DreamAir_On_Flight_Staffs.csv", 'w', newline='') as on_flights_staffs,
      open("DreamAir_Ground_Staffs.csv", 'w', newline='') as off_Flight_staffs):
    job_roles_file.readline().strip('\r\n').split(',')
    print("ID,job_roles_id,name,gender,ph_no,email", file=off_Flight_staffs)
    print("ID,flight_id,job_role_id,name,gender,ph_no,email", file=on_flights_staffs)

    jobs_reader = csv.reader(job_roles_file)
    on_count = 1
    off_count = 1
    for job_role_row in jobs_reader:
        if job_role_row[-1] == 'on':
            for flight_id_no in flight_id_list:
                for i in range(int(job_role_row[-2])):
                    genders = ['Male', 'Female', 'Male']
                    gender = fake.random_element(genders)
                    name = fake.name_male() if gender == "Male" else fake.name_female()
                    e_ph_no = fake_phone_number(fake)
                    e_email = name + e_ph_no[-3:] + "@gmail.com"
                    print(f"{on_count},{flight_id_no},{job_role_row[0]},{name},{gender},{e_ph_no},{e_email}", file=on_flights_staffs)
                    on_count += 1
        else:
            employee_count = int(job_role_row[-2])
            for i in range(employee_count):
                genders = ['Male', 'Female', 'Male']
                gender = fake.random_element(genders)
                name = fake.name_male() if gender == "Male" else fake.name_female()
                e_ph_no = fake_phone_number(fake)
                e_email = name + e_ph_no[-3:] + "@gmail.com"
                print(f"{off_count},{job_role_row[0]},{name},{gender},{e_ph_no},{e_email}", file=off_Flight_staffs)
                off_count += 1

job_roles_df = pd.read_csv("DreamAir_Job_Roles.csv")
job_roles_df.drop("count", inplace=True, axis=1)
job_roles_df.drop("on/off", inplace=True, axis=1)
job_roles_df.to_csv("DreamAir_Job_Roles.csv", index=False)

# Hike dates

In [28]:
hike_dates = hike_dates_list_generator("Hike_Dates.csv")
create_routes("DreamAir_Routes.csv", da_airport_coordinates, da_aircraft_speed, da_price_multiplier, 500)

# Creating necessary dictionaries

In [29]:
all_routes_dict = routes_dict_generator("DreamAir_Routes.csv")
aircraft_dict = aircraft_dict_generator("DreamAir_Aircraft.csv")

# Creating seat info csv file

In [30]:
with open("DreamAir_Seat_Info.csv", 'w', newline='') as seats_file:
    ID = 1
    for flightId, value in aircraft_dict.items():
        max_seats = generate_seat_numbers(int(value))
        for seat_num in max_seats:
            print(f'{ID},{flightId},{seat_num}', file=seats_file)
            ID += 1

# Customer generator and membership updator 
2018 memberships 26 %

2019 memberships 48 %

2020 memberships 36 %

In [31]:
id_2019 = customer_generator("DreamAir_2018_Customers.csv", 275236)
customer_membership_status("DreamAir_2018_Customers.csv", 26)

In [32]:
id_2020 = customer_generator("DreamAir_2019_Customers.csv", 396452, id_2019)
customer_membership_status("DreamAir_2019_Customers.csv", 48)

In [33]:
id_2021 = customer_generator("DreamAir_2020_Customers.csv", 351426, id_2020)
customer_membership_status("DreamAir_2020_Customers.csv", 36)

In [34]:
customer_generator("DreamAir_2021_Customers.csv", 412536, id_2021)
customer_membership_status("DreamAir_2021_Customers.csv", 51)

# Generating Orders 

In [35]:
orders_end_2018, customer_end_2018 = orders_generator(275236, 2018, "DreamAir_2018_Orders.csv", "DreamAir_2018_Schedule.csv", 0, 1)
orders_end_2018_c, customer_end_2018_c = cancellations_updator("DreamAir_2018_Orders.csv", "DreamAir_2018_Cancellations.csv", 5, 275236, 0, 1)

In [36]:
orders_end_2019, customer_end_2019 = orders_generator(396452, 2019, "DreamAir_2019_Orders.csv",
                 "DreamAir_2019_Schedule.csv", customer_end_2018, orders_end_2018)
orders_end_2019_c, customer_end_2019_c = cancellations_updator("DreamAir_2019_Orders.csv", "DreamAir_2019_Cancellations.csv",
                      8, 396452, customer_end_2018_c, orders_end_2018_c)

In [37]:
orders_end_2020, customer_end_2020 = orders_generator(351426, 2020, "DreamAir_2020_Orders.csv",
                 "DreamAir_2020_Schedule.csv", customer_end_2019, orders_end_2019)
orders_end_2020_c, customer_end_2020_c = cancellations_updator("DreamAir_2020_Orders.csv", "DreamAir_2020_Cancellations.csv",
                      10, 351426, customer_end_2019_c, orders_end_2019_c)

In [38]:
orders_generator(351426, 2021, "DreamAir_2021_Orders.csv",
                 "DreamAir_2021_Schedule.csv", customer_end_2020, orders_end_2020)
cancellations_updator("DreamAir_2021_Orders.csv", "DreamAir_2021_Cancellations.csv",
                      6, 351426, customer_end_2020_c, orders_end_2020_c)

(3802731, 1374540)